In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import string

In [ ]:
file = open('test_article.txt', 'r') 

In [ ]:
data = file.readlines()

In [ ]:
data[1]

In [ ]:
sent_tokens = []
for i in data:
    sent_tokens.append(sent_tokenize(i))

In [ ]:
tokens = [sent for sent in map(word_tokenize, data)]

In [ ]:
tokens_lower = [[word.lower() for word in sent]
                 for sent in tokens]

In [ ]:
punctuation_ = set(string.punctuation)

def filter_tokens(sent):
    return([w for w in sent if not w in punctuation_])

In [ ]:
tokens_filtered = list(map(filter_tokens, tokens_lower))
tokens_filtered[0]